In [ ]:
# default_exp metrics

# metrics

> API details.

In [ ]:
#export
import numpy as np
import scipy.stats

In [ ]:
#export
def crps_for_quantiles(probabilistic_forecasts, measurements, quantiles=np.linspace(0.1, 0.9, 9)):
    """ Computes the CRPS score with quantile representation.
        This variant is the variant proposed in Hersbach H. Decomposition of the Continuous Ranked Probability Score for
        Ensemble Prediction Systems. Weather Forecast. 2000;15(5):559-570.
        Parameters
        ----------
            probabilistic_forecasts: array_like
               Either list of "M" scipy.stats.rv_continuous distributions
               https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html
               OR
               2D-numpy array with quantile forecasts with dimensionality M x Q,
               where "Q" is number of quantiles.
            measurements: array_like
               List or numpy array with "M" measurements / observations.
            quantiles: array_like
               List of "Q" values of the quantiles to be evaluated.
        Returns
        -------
            mean_crps: float
                The mean CRIGN over all probabilistic_forecast - measurement pairs.
            single_crps: array, shape (M,)
                CRIGN value for each probabilistic_forecast - measurement pair.
    """

    quantile_forecasts = np.array(probabilistic_forecasts)

    measurements = np.atleast_2d(measurements).T

    alpha_mat = np.diff(np.hstack([quantile_forecasts, measurements]))
    alpha_mat = np.maximum(0, alpha_mat)
    alpha_mat = np.minimum(alpha_mat, np.maximum(0, np.repeat(measurements, quantile_forecasts.shape[1],
                                                              axis=1) - quantile_forecasts))

    beta_mat = np.diff(np.hstack([measurements, quantile_forecasts]))
    beta_mat = np.maximum(0, beta_mat)
    beta_mat = np.minimum(beta_mat,
                          np.maximum(0,
                                     quantile_forecasts - np.repeat(measurements, quantile_forecasts.shape[1], axis=1)))

    single_crps = np.matmul(alpha_mat, np.power(quantiles, 2)) + np.matmul(beta_mat, np.power(quantiles - 1, 2))

    return np.mean(single_crps), single_crps

In [ ]:
quantiles = np.array([0.25,0.5, 0.75])
probabilistic_forecasts = np.array([[1,2,3],[4,5,6],[7,8,9],])
measurements = np.array([2,5,8])

In [ ]:
mean_crps, crps_per_observation = crps_for_quantiles(probabilistic_forecasts, measurements, quantiles)
mean_crps

0.125